In [4]:
import os
os.environ["MANTLE"] = "coreir"
import silica as si
import magma as m
from silica import bits, add, uint, memory
from magma.testing.coroutine import check


@si.coroutine(inputs={"wdata": si.Bits(4), "wen": si.Bit, "ren": si.Bit})
def CombFifo():
    memory = memory(bits(0, 4) for i in range(4))
    raddr = uint(0, 3)
    waddr = uint(0, 3)
    wdata, wen, ren = yield
    while True:
        empty = waddr == raddr
        # full = (waddr[1:0] == raddr[1:0]) & (waddr[2] == ~raddr[2])
        full = and_(and_(waddr[1] == raddr[1], waddr[0] == raddr[0]), waddr[2] != raddr[2])
        rdata = memory[raddr[:2]]
        if wen and not full:
            memory[waddr[:2]] = wdata
            waddr = waddr + 1
        if ren and not empty:
            raddr = raddr + 1
        wdata, wen, ren = yield rdata, empty, full

@si.coroutine(inputs={"wdata": si.Bits(4), "wen": si.Bit, "ren": si.Bit})
def Fifo():
    buffer = memory(4, 4)
    raddr = uint(0, 2)
    waddr = uint(0, 2)
    next_empty = True
    next_full  = False
    wdata, wen, ren = yield
    while True:
        full = next_full
        empty = next_empty
        rdata = buffer[raddr]
        if wen and not full:
            buffer[waddr] = wdata
            waddr = waddr + 1
            next_full = raddr == waddr
            next_empty = False
        if ren and not empty:
            raddr = raddr + 1
            next_empty = raddr == waddr
            next_full = False
        wdata, wen, ren = yield rdata, empty, full
        
fifo = si.compile(Fifo(), file_name="build/silica_fifo.py")
with open("build/silica_fifo.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [2]:
print(repr(fifo))

Fifo = DefineCircuit("Fifo", "empty", Out(Bit), "rdata", Out(Bits(4)), "full", Out(Bit), "wdata", In(Bits(4)), "wen", In(Bit), "ren", In(Bit), "CLK", In(Clock))
inst0 = __silica_BufferFifo()
inst1 = Register2_0001()
next_full = DFF_init0_has_ceFalse_has_resetFalse(name="next_full")
inst3 = coreir_mem4x4()
inst4 = Register2()
next_empty = DFF_init1_has_ceFalse_has_resetFalse(name="next_empty")
inst6 = Register2()
inst7 = not()
inst8 = and_wrapped()
inst9 = Add2()
inst10 = EQ2()
inst11 = not()
inst12 = and_wrapped()
inst13 = Add2()
inst14 = EQ2()
inst15 = fold_and3None()
inst16 = not()
inst17 = and_wrapped()
inst18 = Add2()
inst19 = EQ2()
inst20 = not()
inst21 = and_wrapped()
inst22 = not()
inst23 = fold_and3None()
inst24 = not()
inst25 = and_wrapped()
inst26 = not()
inst27 = not()
inst28 = and_wrapped()
inst29 = Add2()
inst30 = EQ2()
inst31 = fold_and3None()
inst32 = not()
inst33 = and_wrapped()
inst34 = not()
inst35 = not()
inst36 = and_wrapped()
inst37 = not()
inst38 = fold_and3None()

In [3]:
expected_trace = [
        {'wdata': 1, 'wen': 0, 'ren': 1, 'rdata': 0, 'full': False, 'empty': True, 'buffer': [0, 0, 0, 0], 'raddr': 0, 'waddr': 0},
        {'wdata': 2, 'wen': 1, 'ren': 0, 'rdata': 0, 'full': False, 'empty': True, 'buffer': [2, 0, 0, 0], 'raddr': 0, 'waddr': 1},
        {'wdata': 3, 'wen': 1, 'ren': 1, 'rdata': 2, 'full': False, 'empty': False, 'buffer': [2, 3, 0, 0], 'raddr': 1, 'waddr': 2},
        {'wdata': 4, 'wen': 1, 'ren': 0, 'rdata': 3, 'full': False, 'empty': False, 'buffer': [2, 3, 4, 0], 'raddr': 1, 'waddr': 3},
        {'wdata': 5, 'wen': 0, 'ren': 1, 'rdata': 3, 'full': False, 'empty': False, 'buffer': [2, 3, 4, 0], 'raddr': 2, 'waddr': 3},
        {'wdata': 6, 'wen': 0, 'ren': 1, 'rdata': 4, 'full': False, 'empty': False, 'buffer': [2, 3, 4, 0], 'raddr': 3, 'waddr': 3},
        {'wdata': 7, 'wen': 1, 'ren': 0, 'rdata': 0, 'full': False, 'empty': True, 'buffer': [2, 3, 4, 7], 'raddr': 3, 'waddr': 0},
        {'wdata': 8, 'wen': 0, 'ren': 1, 'rdata': 7, 'full': False, 'empty': False, 'buffer': [2, 3, 4, 7], 'raddr': 0, 'waddr': 0},
        {'wdata': 9, 'wen': 1, 'ren': 1, 'rdata': 2, 'full': False, 'empty': True, 'buffer': [9, 3, 4, 7], 'raddr': 0, 'waddr': 1},
        {'wdata': 10, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'buffer': [9, 10, 4, 7], 'raddr': 0, 'waddr': 2},
        {'wdata': 11, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'buffer': [9, 10, 11, 7], 'raddr': 0, 'waddr': 3},
        {'wdata': 12, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': False, 'empty': False, 'buffer': [9, 10, 11, 12], 'raddr': 0, 'waddr': 0},
        {'wdata': 13, 'wen': 1, 'ren': 0, 'rdata': 9, 'full': True, 'empty': False, 'buffer': [9, 10, 11, 12], 'raddr': 0, 'waddr': 0},
        {'wdata': 13, 'wen': 0, 'ren': 1, 'rdata': 9, 'full': True, 'empty': False, 'buffer': [9, 10, 11, 12], 'raddr': 1, 'waddr': 0},
        {'wdata': 14, 'wen': 1, 'ren': 1, 'rdata': 10, 'full': False, 'empty': False, 'buffer': [14, 10, 11, 12], 'raddr': 2, 'waddr': 1},
]

sim_fifo = Fifo()
inputs = ("wdata", "wen", "ren")
outputs = ("rdata", "full", "empty")
states = ("buffer", "raddr", "waddr")
for i, trace in enumerate(expected_trace):
    args = ()
    for input_ in inputs:
        args += (trace[input_], )
    sim_fifo.send(args)
    for output in outputs:
        assert getattr(sim_fifo, output) == trace[output], (i, output,  getattr(sim_fifo, output), trace[output])
    for state in states:
        if state in ["waddr", "raddr"]:
            assert getattr(sim_fifo, state)[:2] == trace[state], (i, state,  getattr(sim_fifo, state), trace[state])
        else:
            assert getattr(sim_fifo, state) == trace[state], (i, state,  getattr(sim_fifo, state), trace[state])

@si.coroutine
def inputs_generator(N):
    while True:
        for trace in expected_trace:
            wdata = bits(trace["wdata"], N)
            wen = bool(trace["wen"])
            ren = bool(trace["ren"])
            yield wdata, wen, ren
        
check(fifo, Fifo(), len(expected_trace), inputs_generator(4))
print("Passed!")

MagmaTransformException: Calling `.value()` on ren returned None. Likely an unwired port.

In [ ]:
!magma -o coreir -m coreir -t Fifo build/silica_fifo.py
!coreir -i build/silica_fifo.json -o build/silica_fifo.v
# !magma -o verilog -m lattice -t Fifo build/silica_fifo.py
!yosys -p 'synth_ice40 -top Fifo -blif build/silica_fifo.blif' build/silica_fifo.v | grep -A 20 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_fifo.txt build/silica_fifo.blif
!icetime -tmd hx1k build/silica_fifo.txt  | grep -B 2 "Total path delay"

In [ ]:
!yosys -p 'synth_ice40 -top fifo -blif build/verilog_fifo.blif' ../verilog/fifo.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_fifo.txt build/verilog_fifo.blif
!icetime -tmd hx1k build/verilog_fifo.txt | grep -B 2 "Total path delay"

In [ ]:
import magma
from magma.testing.verilator import compile as compileverilator
from magma.testing.verilator import run_verilator_test
from magma.bit_vector import BitVector

def check_verilog(circ, circ_sim, num_cycles, inputs_generator=None):
    test_vectors = magma.testing.coroutine.testvectors(circ, circ_sim, num_cycles,
            inputs_generator if inputs_generator else None)

    name = "fifo"
    with open(f"build/sim_{name}_verilator.cpp", "w") as verilator_harness:
        verilator_harness.write(f'''\
#include "V{name}.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

void check(const char* port, int a, int b, int cycle) {{
    if (!(a == b)) {{
        std::cerr << \"Got      : \" << a << std::endl;
        std::cerr << \"Expected : \" << b << std::endl;
        std::cerr << \"Cycle    : \" << cycle << std::endl;
        std::cerr << \"Port     : \" << port << std::endl;
        exit(1);
    }}
}}

int main(int argc, char **argv, char **env) {{
    Verilated::commandArgs(argc, argv);
    V{name}* top = new V{name};
''')
    
        cycle = 0
        for inputs, outputs in zip(test_vectors[0], test_vectors[1]):
            print(cycle)
            print(inputs) 
            print(outputs)
            for port_name, value in inputs.items():
                if isinstance(value, list):
                    string = ""
                    for elem in value:
                        string = elem.as_binary_string()[2:] + string
                    value = "0b" + string
                elif isinstance(value, BitVector):
                    value = value.as_int()
                else:
                    value = int(value)
                verilator_harness.write("    top->{} = {};\n".format(port_name, value))
            for port_name, value in outputs.items():
                if isinstance(value, BitVector):
                    value = value.as_binary_string()
                else:
                    value = int(value)
                verilator_harness.write("    top->eval();\n")
                verilator_harness.write("    check(\"{port_name}\", top->{port_name}, {expected}, {cycle});\n".format(port_name=port_name, expected=value, cycle=cycle))

            verilator_harness.write("    top->CLK = 0;\n")
            verilator_harness.write("    top->eval();\n")
            verilator_harness.write("    top->CLK = 1;\n")
            verilator_harness.write("    top->eval();\n")
            cycle += 1
        verilator_harness.write("}\n")

    run_verilator_test(
        "fifo",
        f"sim_{name}_verilator",
        name,
        "-I../../verilog"
    )

check_verilog(fifo, Fifo(), len(expected_trace), inputs_generator(4))
print("Passed!")